In [7]:
import pandas as pd
import unittest
import numpy as np
np.random.seed(1)

In [9]:
testData = {'Area': (['Indonesia'] * 3 + ['China'] * 3 + ['UK'] * 3) * 3,
            'Item': ['Apples'] * 9 + ['Grapes'] * 9 + ['Beehives'] * 9,
            'Element': ['Yield', 'Production', 'Area'] * 6 + ['Yield', 'Stocks', 'Count'] * 3,
           '2019': np.random.randint(100, 100000, size = 27), '2020': np.random.randint(100, 100000, size = 27)}

testDf = pd.DataFrame(testData)
testDf.head()

,Area,Item,Element,2019,2020
0,Indonesia,Apples,Yield,19533,34480
1,Indonesia,Apples,Production,17237,85118
2,Indonesia,Apples,Area,26203,1944
3,China,Apples,Yield,44610,9652
4,China,Apples,Production,51140,35537


In [10]:
def itemToProductionElementDictionary(agDF):
    """
        Create a dictionary mapping each item
        to its production element. 
        Args:
            agDF: Agriculture dataframe from FAO
        Returns: a dictionary similar to {'Apples': 'Production', 'Beehives': 'Stocks'}
    """
    
    productionElements = ["Production", "Stocks", "Laying", "Prod Popultn"]
    
    dfProdElements = agDF[agDF['Element'].isin(productionElements)]
    itemsList = dfProdElements['Item']
    elementsList = dfProdElements['Element']
    
    itemProdElementDict = {}
    
    for itemElementTuple in zip(itemsList, elementsList):
        itemProdElementDict[itemElementTuple[0]] = itemElementTuple[1]
    
    return itemProdElementDict

In [17]:
class TestItemToProdElement(unittest.TestCase):
    def test_itemToProductionElementDictionary(self):
        testData = {'Area': (['Indonesia'] * 3 + ['China'] * 3 + ['UK'] * 3) * 3,
            'Item': ['Apples'] * 9 + ['Grapes'] * 9 + ['Beehives'] * 9,
            'Element': ['Yield', 'Production', 'Area'] * 6 + ['Yield', 'Stocks', 'Count'] * 3,
           '2019': np.random.randint(100, 100000, size = 27), '2020': np.random.randint(100, 100000, size = 27)}
        testDf = pd.DataFrame(testData)
        itemToProdElementDict = itemToProductionElementDictionary(testDf)
        expected = {'Apples': 'Production', 'Grapes': 'Production', 'Beehives': 'Stocks'}
        self.assertEqual(expected, itemToProdElementDict)

In [18]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_itemToProductionElementDictionary (__main__.TestItemToProdElement) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.037s

OK
